In [ ]:
import sys

In [ ]:
sys.path.append("D:\\API-AnomalyFinder-Multivariate\\cases\\PBSF\\")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, f1_score, roc_curve, precision_score, recall_score
import lightgbm as lgb
from sklearn.svm import SVC
%matplotlib inline

In [ ]:
def validate_models(name, method, model_type):
    with open(f"../features/20210512/{method}/{name}/features.pkl", "rb") as f:
        data = pickle.load(f)
    train_features = data["train"]["features"]
    train_labels = data["train"]["labels"]
    test_features = data["test"]["features"]
    test_labels = data["test"]["labels"]
    if model_type == "lgbm":
        model = lgb.LGBMClassifier(num_leaves=5, n_estimators=150, learning_rate=0.05)
    elif model_type == "svm":
        model = SVC(gamma="auto", probability=True)
    model.fit(train_features, train_labels)
    proba = model.predict_proba(test_features)
    proba_train = model.predict_proba(train_features)
    auc = roc_auc_score(test_labels, proba[:, 1])
    _, _, thresholds = roc_curve(test_labels, proba[:, 1])
    best_f1 = 0
    for th in thresholds:
        pred_th = np.zeros_like(proba[:, 1])
        pred_th[proba[:, 1] > th] = 1
        f1 = f1_score(test_labels, pred_th)
        best_f1 = max(best_f1, f1)
    result = {
        "name": name,
        "best_f1": best_f1,
        "auc": auc
    }
    return result

In [ ]:
exams = {
    "patient_id_1": {
        "method": "moco",
        "model_type": "svm"
    },
    "patient_id_2": {
        "method": "contrastive",
        "model_type": "svm"
    }
}

In [ ]:
results = []
for name, config in exams.items():
    results.append(
        validate_models(name, config["method"], config["model_type"])
    )

In [ ]:
for x in results:
    print(f"{x['name']}\tF1 score: {x['best_f1']:.2%}\tAUC score: {x['auc']:.3}")